In [1]:
from API.get_data.api_yahoo import get_options

In [2]:
options = get_options("TSLA")

    call → Une option CALL donne à l'acheteur le droit (mais pas l'obligation) d'ACHETER l'actif sous-jacent à un prix prédéfini avant ou à la date d'expiration
    put → Une option PUT donne à l'acheteur le droit (mais pas l'obligation) de VENDRE l'actif sous-jacent à un prix prédéfini avant ou à la date d'expiration

In [38]:
def get_stat(df, col):
    min = df[col].min()
    max = df[col].max()
    mean = df[col].mean()
    median = df[col].median()
    return min, max, mean, median

In [3]:
from API.get_data.api_yahoo import get_analyst_price_targets

In [58]:
current_price = get_analyst_price_targets("TSLA")["current"]

In [36]:
calls = options[(options["Type"]=="calls") & (options["ask"]>0)]

In [98]:
# Moyenne pondérée par le volume
# moyenen pondere = somme(ak_price * volume) / somme des volumes
# but refléter l'importance relative de chaque option en prenant en compte les volumes
mean_weighted_ask = (calls["ask"] * calls["volume"]).sum() / calls["volume"].sum()
mean_weighted_bid = (calls["bid"] * calls["volume"]).sum() / calls["volume"].sum()

# La différence entre prix moyen du vendeur et prix moyen que vendeur pret à mettre 
# weighted_spread = mean_weighted_ask - mean_weighted_bid

In [116]:
CLASSIFICATION_RANGES = [
    (-float("inf"), -25, "Very undervalued"), # Très sous-évalué
    (-25, -5, "Slightly undervalued"), # Légèrement sous-évalué
    (-5, 5, "Price within spread"), # Prix dans le spread
    (5, 25, "Slightly overvalued"), # Légèrement surévalué
    (25, float("inf"), "Very overvalued"), #Très surévalué
]

def get_ratio_cluster(ratio_spread):
    """
    Estimate the cluster of the ratio_spread according to the defined intervals.

    Args:
        ratio_spread (float): Le ratio_spread à classifier.

    Returns:
        str: La classification correspondant au ratio_spread.
    """
    for lower_value, upper_value, cluster in CLASSIFICATION_RANGES:
        if lower_value <= ratio_spread < upper_value:
            return cluster
    return "Non classifié"  # Cas par défaut (ne devrait pas arriver)
    

In [117]:
spread_radio = ((mean_weighted_ask - mean_weighted_bid) / (mean_weighted_ask + mean_weighted_bid)) * 100

get_ratio_cluster(spread_radio)

'Price within spread'

In [19]:
CLASSIFICATION_RANGES = [
    (-float("inf"), -25, "Very undervalued"), # Très sous-évalué
    (-25, -5, "Slightly undervalued"), # Légèrement sous-évalué
    (-5, 5, "Price within spread"), # Prix dans le spread
    (5, 25, "Slightly overvalued"), # Légèrement surévalué
    (25, float("inf"), "Very overvalued"), #Très surévalué
]

def classify_spread(spread_ratio):
    """
    Classify the spread ratio into predefined clusters.

    Args:
        spread_ratio (float): The spread ratio to classify.

    Returns:
        str: The cluster classification.
    """
    return next(
        (cluster for lower, upper, cluster in CLASSIFICATION_RANGES if lower <= spread_ratio < upper),
        "Non classifié",  # Default case
    )


In [20]:
def get_options_cluster(df):
    """
    Determine the market expectations of the company via these options. Those options can be :
        - Very undervalued
        - Slightly undervalued
        - Price within spread
        - Slightly overpriced
        - Very overpriced
    If is undervalued → sould buy
    If is overvalued → sould not buy

    Args:
        df (dataframe): the dataframe to classify

    Returns:
        str: the cluster.
    """

    # Weighted average by option volume
    # weighted average = sum(price * volume) / sum of volumes
    # goal is to reflect the relative importance of each option by taking volumes into account
    mean_weighted_ask = (df["ask"] * df["volume"]).sum() / df["volume"].sum()
    mean_weighted_bid = (df["bid"] * df["volume"]).sum() / df["volume"].sum()

    spread_ratio = ((mean_weighted_ask - mean_weighted_bid) / (mean_weighted_ask + mean_weighted_bid)) * 100

    # for lower_value, upper_value, cluster in CLASSIFICATION_RANGES:
    #     if lower_value <= spread_ratio < upper_value:
    #         return cluster
    # return "Non classifié"  # Cas par défaut (ne devrait pas arriver)
    return classify_spread(spread_ratio)
    

In [ ]:
faire un cluster pour call et un cluster pour puts
enlever les ann pour eviter les div par 0

In [21]:
calls = options[(options["Type"]=="calls") & (options["ask"]>0)]
get_options_cluster(calls)

'Price within spread'

In [17]:
puts = options[(options["Type"]=="puts") & (options["ask"]>0)]
get_options_cluster(puts)

'Price within spread'

In [12]:
df = options[(options["Type"]=="puts") & (options["ask"]>0)]
mean_weighted_ask = (df["ask"] * df["volume"]).sum() / df["volume"].sum()
mean_weighted_bid = (df["bid"] * df["volume"]).sum() / df["volume"].sum()

print(mean_weighted_ask, "-", mean_weighted_bid)

14.354896927074227 - 14.024458907445654


In [13]:
df = options[(options["Type"]=="calls") & (options["ask"]>0)]
mean_weighted_ask = (df["ask"] * df["volume"]).sum() / df["volume"].sum()
mean_weighted_bid = (df["bid"] * df["volume"]).sum() / df["volume"].sum()

print(mean_weighted_ask, "-", mean_weighted_bid)

17.255944721654576 - 16.898941187038243


In [10]:
get_options_cluster(options)

'Price within spread'

In [ ]:
def 

moyenen pondere 
mediane price
si sur ou sous evalue 

In [49]:
mean_ask

np.float64(121.92125000000001)

In [44]:
min_ask, max_ask, mean_ask, median_ask = get_stat(calls, "ask")
min_bid, max_bid, mean_bid, median_bid = get_stat(calls, "bid")

diff_min = min_ask - min_bid
diff_max = max_ask - max_bid
diff_mean = mean_ask - mean_bid
diff_median = median_ask - median_bid

print(diff_min, diff_max, diff_mean, diff_median)

0.01 6.25 1.826708185053377 1.1749999999999972


Calcul de l'écart entre prix acheteur et vendeur
    Si le décalage entre le prix proposé par le vendeur et celui proposé par l'acheteur est important, on peut considère que le prix demander ne correspond pas aux attentes du marché.
    
    Cas 1 : Valeur positive (acheteur demande trop)
        Si le prix proposé par l'acheteur est supérieur → option est surévaluée.

    Cas 2 : Valeur négative (beaucoup de demandes)
        Si la demande est forte (prix acheteur élevé par rapport au vendeur) → option est sous-évaluée par le marché à cet instant.

In [ ]:
def spread_bid_ask()
    """
    Calculating the gap between buyer and seller prices
    If the gap between the price proposed by the seller and that proposed by the buyer is significant, we can consider that the asking price does not correspond to market expectations.
    
    Case 1: Positive value (buyer asks too much)
        If the price proposed by the buyer is higher → option is overvalued.

    Case 2: Negative value (many requests)
        If demand is strong (buyer's price high relative to seller's) → option is undervalued by the market at that moment.
    """

In [27]:
options[options["Type"]=="calls"].sort_values("ask")

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expirationDate,Type
2057,TSLA250620C00040000,2024-12-02 19:42:44+00:00,40.0,318.56,0.00,0.00,0.000000,0.000000,8.0,0,0.000010,True,REGULAR,USD,2025-06-20,calls
3150,TSLA251219C00035000,2024-12-02 20:23:21+00:00,35.0,326.62,0.00,0.00,0.000000,0.000000,1.0,0,0.000010,True,REGULAR,USD,2025-12-19,calls
151,TSLA250124C00750000,2025-01-17 20:58:22+00:00,750.0,0.01,0.00,0.01,0.000000,0.000000,86.0,1213,1.156254,False,REGULAR,USD,2025-01-24,calls
159,TSLA250124C00830000,2025-01-17 16:04:54+00:00,830.0,0.01,0.00,0.01,0.000000,0.000000,1.0,166,1.343753,False,REGULAR,USD,2025-01-24,calls
165,TSLA250124C00890000,2025-01-15 17:29:00+00:00,890.0,0.01,0.00,0.01,0.000000,0.000000,19.0,542,1.468753,False,REGULAR,USD,2025-01-24,calls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3144,TSLA251219C00005000,2025-01-16 18:02:38+00:00,5.0,409.75,419.90,424.70,0.000000,0.000000,101.0,4374,2.500004,True,REGULAR,USD,2025-12-19,calls
3379,TSLA260116C00005000,2025-01-17 19:20:05+00:00,5.0,433.50,418.35,426.65,25.500000,6.250000,9.0,11694,2.521488,True,REGULAR,USD,2026-01-16,calls
3146,TSLA251219C00015000,2024-11-14 16:15:40+00:00,15.0,304.26,418.90,426.90,0.000000,0.000000,10.0,31,3.922363,True,REGULAR,USD,2025-12-19,calls
3598,TSLA260618C00005000,2025-01-16 20:51:43+00:00,5.0,410.56,419.05,426.90,0.000000,0.000000,11.0,1896,2.329594,True,REGULAR,USD,2026-06-18,calls


In [21]:
options[options["Type"]=="calls"]["ask"].mean()

np.float64(121.81287555555555)

In [22]:
options[options["Type"]=="calls"]["ask"].median()

np.float64(81.65)

In [25]:
options[options["Type"]=="calls"]["ask"].max()

np.float64(427.1)

In [23]:
options[options["Type"]=="calls"]["ask"].min()

np.float64(0.0)

In [ ]:
def count_options():
    nbr_puts = options[options["Type"]=="puts"].shape[0]
    nbr_calls = options[options["Type"]=="calls"].shape[0]